<a href="https://colab.research.google.com/github/tonyhk9528/Algomap-leetcode/blob/main/Cardiff_NLP_Hackathon_2025_Starter_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cardiff NLP Hackathon 2025 - Starter Code

Welcome to Cardiff NLP's second hackathon! Below is some code to get started on the AMPLYFI API and look at some data.

====================

Note: the API is a real time resource so extra points to projects that can treat it as a continual data stream rather than a one-off data source!

Another thing to note about this is that it will affect Amplyfi's servers if you download a silly amount of data. We ask that you only request 100 results per request, but if you have the data you need, try to download it or store it as a variable rather than requesting the exact same data over and over again.

In [ ]:
# Import some libraries

import requests
import json
import nltk
import re
import pandas as pd

from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

Amplyfi have provided some limits and explanations of what you can query the API for below:

`query_text` anything

`result_size` <=100

`include_highlights` boolean (if True, you get sentences matching keyphrases in the query)

`include_smart_tags` boolean (if True, you get back metadata from our "smart" market intel classifiers - signals and industries)

`ai_answer` can only accept "basic", this will take the 5 most relevant docs and answer the query_text based on them




In [ ]:
# API endpoint from the newly deployed service

API_URL = "https://zfgp45ih7i.execute-api.eu-west-1.amazonaws.com/sandbox/api/search"
API_KEY = "GHJ38746G38B7RB46GBER"

headers = {
    "Content-Type": "application/json",
    "x-api-key": API_KEY
}

# Edit the below to get different data
payload = {
  "query_text": "What has Boeing been doing recently?",
  "result_size": 10,
  "include_highlights":True,
  "ai_answer": "basic"
}

response = requests.post(API_URL, headers=headers, data=json.dumps(payload))
json_response = response.json()

json_response


{'message': 'Endpoint request timed out'}

In [ ]:
json_response['results'][0]

KeyError: 'results'

In [ ]:
df = pd.json_normalize(json_response['results'])

df.head()

## Example Sentiment Analysis

In [ ]:
## Clean data

def clean_text(text):
    """
    - Convert to lowercase
    - Remove URLs
    - Remove punctuation / non-alpha
    - Collapse multiple spaces
    """
    if not isinstance(text, str):
        return ""
    # Remove URLs (very basic)
    text = re.sub(r"http\S+|www\.\S+", "", text)
    # Lowercase
    text = text.lower()
    # Keep only letters and spaces
    text = re.sub(r"[^a-z\s]", " ", text)
    # Collapse multiple spaces
    text = re.sub(r"\s+", " ", text).strip()
    return text

df['clean_summary'] = df['summary'].apply(clean_text)


In [ ]:
## Sentiment analysis example

sia = SentimentIntensityAnalyzer()

def get_sentiment_scores(text):
    """
    Returns a dict with these keys:
       - neg: negative sentiment score
       - neu: neutral score
       - pos: positive score
       - compound: normalized, weighted composite (-1 to +1)
    """
    return sia.polarity_scores(text)

# Apply to each summary
df['sentiment'] = df['clean_summary'].apply(get_sentiment_scores)

# Split into separate columns if you like
df['sent_neg'] = df['sentiment'].apply(lambda d: d['neg'])
df['sent_neu'] = df['sentiment'].apply(lambda d: d['neu'])
df['sent_pos'] = df['sentiment'].apply(lambda d: d['pos'])
df['sent_compound'] = df['sentiment'].apply(lambda d: d['compound'])

# Quick look at top 5 compound scores
print(df[['clean_summary', 'sent_compound']].sort_values(by='sent_compound', ascending=False).head())
print(df[['clean_summary', 'sent_compound']].sort_values(by='sent_compound').head())


In [ ]:
# Find index of the most positive (max compound) and most negative (min compound) summaries
max_idx = df['sent_compound'].idxmax()
min_idx = df['sent_compound'].idxmin()

# Retrieve the scores
max_score = df.loc[max_idx, 'sent_compound']
min_score = df.loc[min_idx, 'sent_compound']

# Print the full clean summaries along with their sentiment scores
print("Most positive summary (compound = {:.3f}):\n".format(max_score))
print(df.loc[max_idx, 'clean_summary'])


print("\n\nMost negative summary (compound = {:.3f}):\n".format(min_score))
print(df.loc[min_idx, 'clean_summary'])

## Project Ideas

Feel free to use this code to start your own project, and here are some (Chat-GPT generated 😬) ideas for projects:

* Real-Time Sentiment Pulse: Visualize sentiment trends over the past 24-48 hours for any keyword.

* One-Click News Brief: Generate a 3-sentence summary of today's top articles on a given topic.

* Bias/Slant Detector: Compare headlines from multiple outlets on the same event and label their bias.

* Event Timeline Generator: Autofill a chronological list of key dates and summaries for any query.

* Breaking News Alert Bot: Push a short alert whenever article volume spikes or sentiment turns extreme.

* Multilingual Hashtag Trend Mapper: Show related hashtags and translations across different languages.

* Rumor vs. Fact Checker: Verify a user-provided statement against recent reputable sources.

* “What's Changed?” Comparator: Highlight how coverage of a topic has shifted from last month to last week.

* Geo-Mood Map: Color-code countries by average sentiment or topic intensity on a query.

* Voice-Activated News Q&A: Let users speak a question and hear back a 2–3 sentence summary of current events.

## Dashboard libraries for Python

https://shiny.posit.co/py/

https://dash.plotly.com/

https://streamlit.io/

